# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 24 2022 12:33PM,237979,20,ACI-8052349,"ACI Healthcare USA, Inc.",Released
1,May 24 2022 12:33PM,237979,20,ACI-8052353,"ACI Healthcare USA, Inc.",Released
2,May 24 2022 12:33PM,237979,20,8052413,"ACI Healthcare USA, Inc.",Released
3,May 24 2022 12:33PM,237979,20,8052416,"ACI Healthcare USA, Inc.",Released
4,May 24 2022 12:33PM,237979,20,8052417,"ACI Healthcare USA, Inc.",Released
5,May 24 2022 12:33PM,237979,20,8052419,"ACI Healthcare USA, Inc.",Released
6,May 24 2022 12:33PM,237979,20,8052479,"ACI Healthcare USA, Inc.",Released
7,May 24 2022 12:33PM,237979,20,8052480,"ACI Healthcare USA, Inc.",Released
8,May 24 2022 12:33PM,237979,20,8052482,"ACI Healthcare USA, Inc.",Released
9,May 24 2022 12:33PM,237979,20,8052491,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,237975,Released,19
38,237976,Released,1
39,237977,Released,1
40,237978,Released,8
41,237979,Released,19


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237975,NaN,NaN,19.0
237976,NaN,NaN,1.0
237977,NaN,NaN,1.0
237978,NaN,NaN,8.0
237979,NaN,NaN,19.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237701,0.0,0.0,1.0
237856,0.0,0.0,1.0
237864,0.0,0.0,11.0
237878,0.0,13.0,8.0
237883,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237701,0,0,1
237856,0,0,1
237864,0,0,11
237878,0,13,8
237883,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237701,0,0,1
1,237856,0,0,1
2,237864,0,0,11
3,237878,0,13,8
4,237883,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237701,,,1
1,237856,,,1
2,237864,,,11
3,237878,,13,8
4,237883,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 24 2022 12:33PM,237979,20,"ACI Healthcare USA, Inc."
19,May 24 2022 12:32PM,237978,20,Reliable 1 Laboratories LLC
27,May 24 2022 12:23PM,237977,10,Therapeutics Retain 2
28,May 24 2022 12:17PM,237976,10,Emerginnova
29,May 24 2022 12:16PM,237975,19,"AdvaGen Pharma, Ltd"
48,May 24 2022 12:14PM,237974,10,Aerogen LTD.
53,May 24 2022 11:51AM,237973,10,"Citieffe, Inc."
54,May 24 2022 11:46AM,237856,18,Hush Hush
55,May 24 2022 11:37AM,237971,15,"Alexso, Inc."
56,May 24 2022 11:35AM,237967,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 24 2022 12:33PM,237979,20,"ACI Healthcare USA, Inc.",,,19
1,May 24 2022 12:32PM,237978,20,Reliable 1 Laboratories LLC,,,8
2,May 24 2022 12:23PM,237977,10,Therapeutics Retain 2,,,1
3,May 24 2022 12:17PM,237976,10,Emerginnova,,,1
4,May 24 2022 12:16PM,237975,19,"AdvaGen Pharma, Ltd",,,19
5,May 24 2022 12:14PM,237974,10,Aerogen LTD.,,,5
6,May 24 2022 11:51AM,237973,10,"Citieffe, Inc.",,,1
7,May 24 2022 11:46AM,237856,18,Hush Hush,,,1
8,May 24 2022 11:37AM,237971,15,"Alexso, Inc.",,,1
9,May 24 2022 11:35AM,237967,10,Bio-PRF,,,18


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 12:33PM,237979,20,"ACI Healthcare USA, Inc.",19,,
1,May 24 2022 12:32PM,237978,20,Reliable 1 Laboratories LLC,8,,
2,May 24 2022 12:23PM,237977,10,Therapeutics Retain 2,1,,
3,May 24 2022 12:17PM,237976,10,Emerginnova,1,,
4,May 24 2022 12:16PM,237975,19,"AdvaGen Pharma, Ltd",19,,
5,May 24 2022 12:14PM,237974,10,Aerogen LTD.,5,,
6,May 24 2022 11:51AM,237973,10,"Citieffe, Inc.",1,,
7,May 24 2022 11:46AM,237856,18,Hush Hush,1,,
8,May 24 2022 11:37AM,237971,15,"Alexso, Inc.",1,,
9,May 24 2022 11:35AM,237967,10,Bio-PRF,18,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 12:33PM,237979,20,"ACI Healthcare USA, Inc.",19,,
1,May 24 2022 12:32PM,237978,20,Reliable 1 Laboratories LLC,8,,
2,May 24 2022 12:23PM,237977,10,Therapeutics Retain 2,1,,
3,May 24 2022 12:17PM,237976,10,Emerginnova,1,,
4,May 24 2022 12:16PM,237975,19,"AdvaGen Pharma, Ltd",19,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 12:33PM,237979,20,"ACI Healthcare USA, Inc.",19.0,NaN,NaN
1,May 24 2022 12:32PM,237978,20,Reliable 1 Laboratories LLC,8.0,NaN,NaN
2,May 24 2022 12:23PM,237977,10,Therapeutics Retain 2,1.0,NaN,NaN
3,May 24 2022 12:17PM,237976,10,Emerginnova,1.0,NaN,NaN
4,May 24 2022 12:16PM,237975,19,"AdvaGen Pharma, Ltd",19.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 12:33PM,237979,20,"ACI Healthcare USA, Inc.",19.0,0.0,0.0
1,May 24 2022 12:32PM,237978,20,Reliable 1 Laboratories LLC,8.0,0.0,0.0
2,May 24 2022 12:23PM,237977,10,Therapeutics Retain 2,1.0,0.0,0.0
3,May 24 2022 12:17PM,237976,10,Emerginnova,1.0,0.0,0.0
4,May 24 2022 12:16PM,237975,19,"AdvaGen Pharma, Ltd",19.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2379652,93.0,28.0,9.0
12,237883,0.0,1.0,0.0
15,1189734,34.0,27.0,18.0
16,951698,4.0,9.0,0.0
17,237924,1.0,0.0,0.0
18,951459,4.0,0.0,0.0
19,713852,21.0,0.0,0.0
20,951699,46.0,13.0,0.0
22,237907,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2379652,93.0,28.0,9.0
1,12,237883,0.0,1.0,0.0
2,15,1189734,34.0,27.0,18.0
3,16,951698,4.0,9.0,0.0
4,17,237924,1.0,0.0,0.0
5,18,951459,4.0,0.0,0.0
6,19,713852,21.0,0.0,0.0
7,20,951699,46.0,13.0,0.0
8,22,237907,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,93.0,28.0,9.0
1,12,0.0,1.0,0.0
2,15,34.0,27.0,18.0
3,16,4.0,9.0,0.0
4,17,1.0,0.0,0.0
5,18,4.0,0.0,0.0
6,19,21.0,0.0,0.0
7,20,46.0,13.0,0.0
8,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,93.0
1,12,Released,0.0
2,15,Released,34.0
3,16,Released,4.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,22
Status,,,,,,,,,
Completed,9.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,28.0,1.0,27.0,9.0,0.0,0.0,0.0,13.0,1.0
Released,93.0,0.0,34.0,4.0,1.0,4.0,21.0,46.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,22
0,Completed,9.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,1.0,27.0,9.0,0.0,0.0,0.0,13.0,1.0
2,Released,93.0,0.0,34.0,4.0,1.0,4.0,21.0,46.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,22
0,Completed,9.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,1.0,27.0,9.0,0.0,0.0,0.0,13.0,1.0
2,Released,93.0,0.0,34.0,4.0,1.0,4.0,21.0,46.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()